In [1]:
from mongo_processor.NeuroscienceNewsMongoConnector import NeuroscienceNewsMongoConnector

In [2]:
from neuroscience_news.NeuroscienceNewsScraper import NeuroscienceNewsArticleScraper

In [3]:
from joblib import Parallel, delayed

In [ ]:
articles_file = "./data/neuroscience_news_urls.txt"

In [ ]:
with open(articles_file) as file:
    articles = file.read().splitlines()

In [ ]:
db_name = "Neuroscience_News"
sc = NeuroscienceNewsArticleScraper()
mc = NeuroscienceNewsMongoConnector(db_name)

In [ ]:
scraped_articles = Parallel(n_jobs=-1)(delayed(sc.scrape_article)(article) for article in articles)

In [ ]:
mc.save_neuroscience_articles(scraped_articles)